# Computation Notebook 

This notebook is meant to show and compare the computational costs with the grid-based approach (searching all possible gridpoints) versus the gridless approach (binary search) to find decision boundary points. 

We show the times that are associated with each of these.

In [1]:
%load_ext cuml.accel
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
#import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import warnings
import random 
from sklearn.utils import resample

random.seed(0)
warnings.filterwarnings('ignore', category=UserWarning)

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext cuml.accel
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
#import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import warnings
import random 
from sklearn.utils import resample

random.seed(0)
warnings.filterwarnings('ignore', category=UserWarning)

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

The cuml.accel extension is already loaded. To reload it, use:
  %reload_ext cuml.accel


# Datasets with a varying number of features ($2$, $10$, $50$) and a model (Logistic Regression) 

We construct a set of pandas dataframes (df1, df2, df3) and a model logistic regression alongside the dataset.

In [3]:
X, y = make_classification(n_samples=2000, n_features=2, n_informative=2, n_redundant=0, random_state=42, n_classes=2)
model = LogisticRegression()
y = y.reshape(-1,1)
df1 = pd.DataFrame(data=np.hstack((X,y)))

In [4]:
X, y = make_classification(n_samples=2000, n_features=10, n_informative=10, n_redundant=0, random_state=42, n_classes=2)
y = y.reshape(-1,1)
df2 = pd.DataFrame(data=np.hstack((X,y)))

In [5]:
X, y = make_classification(n_samples=2000, n_features=50, n_informative=50, n_redundant=0, random_state=42, n_classes=2)
y = y.reshape(-1,1)
df3 = pd.DataFrame(data=np.hstack((X,y)))

# Experiments of the Computation Costs for Grid-based Approach (Logistic Regression with Numba)

$\textbf{ Logistic Regression with Grid-based Method }$ -- 50 features

Resolution: $R = 15$ ($15^{50}$ points), Memory Error (Maximum allowed dimension exceeded), 0 boundary points found

Resolution: $R = 10$ ($10^{50}$ points), Memory Error (Maximum allowed dimension exceeded), 0 boundary points found

$\textbf{ Logistic Regression with Grid-based Method }$ -- 10 features

Resolution: $R = 15$ ($15^{15}$ points), 42 Terabyte Memory Error, 0 boundary points found

Resolution: $R = 10$ ($10^{10}$ points), 745 Gigabytes Memory Error, 0 boundary points found


$\textbf{ Logistic Regression with Grid-based Method }$ -- 2 features

Resolution: $R = 150$ ($150^2 = 22,500$ points searched), 28.1 seconds runtime, 454 boundary points found

Resolution: $R = 100$ ($100^2 = 10,000$ points searched) 5.5 seconds runtime, 104 boundary points found

In [6]:
from files.grid_optimal_point import optimal_point

In [7]:
X, y = make_classification(n_samples=2000, n_features=2, n_informative=2, n_redundant=0, random_state=42, n_classes=2)
model = LogisticRegression()
y = y.reshape(-1,1)
df1 = pd.DataFrame(data=np.hstack((X,y)))

In [8]:
df1.head(n=10)

,0,1,2
0,0.800062,-0.957489,1.0
1,1.187099,1.159787,1.0
2,0.154512,1.217520,0.0
3,0.179014,-0.852832,1.0
4,-0.735827,-0.245366,0.0
5,0.039487,1.320957,1.0
6,-1.482199,0.419738,0.0
7,-0.622829,-0.803223,0.0
8,0.965721,-1.068587,1.0
9,0.798459,-1.022348,1.0


In [9]:
optimal_point(df1, model=model, desired_class=0, original_class=1, resolution=150, chosen_row=1, point_epsilon=0.1, epsilon=0.07, plot=False)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
Number of boundary points
(311, 2)
Finding the closest point from the contour line to the point...
Finding the closest point from the contour line to the point.
[[1.07092306 0.87624128]]
[[-1.16018181  0.00452233]]


[np.float64(-0.05536678064834133), np.float64(1.163749815503608)]

In [10]:
optimal_point(df1, model=model, desired_class=0, original_class=1, resolution=100, chosen_row=1, point_epsilon=0.1, epsilon=0.07)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
Number of boundary points
(104, 2)
Finding the closest point from the contour line to the point...
Finding the closest point from the contour line to the point.
[[1.07092411 1.09801295]]
[[-1.15887253 -0.03919662]]


[np.float64(-0.05396585561956102), np.float64(1.116748769377007)]

In [11]:
df2.head(n=10)

,0,1,2,3,4,5,6,7,8,9,10
0,1.990620,-2.921959,-5.689956,3.173745,-4.393239,0.538239,-0.962650,-3.024594,-0.837913,-1.944185,1.0
1,1.342356,0.942957,3.057782,0.059667,-0.611317,-0.985521,0.323851,0.853719,1.741036,2.069323,1.0
2,-1.063987,2.185873,-2.230002,1.095676,-0.240747,-1.776678,-4.165090,-0.122159,0.688403,1.401783,1.0
3,-4.852159,1.297380,3.466596,4.114327,-4.004815,-0.591707,2.035825,3.118117,-0.996438,1.465570,1.0
4,0.033975,-0.181000,0.438378,-1.506810,-0.503984,2.442108,-0.408010,1.722336,2.563095,-3.228898,1.0
5,2.770680,-0.667671,-2.726522,1.778277,3.033320,1.052492,1.810689,-3.730430,-2.032935,2.371080,0.0
6,-1.375939,0.889744,-0.487178,0.394568,-1.553368,2.302102,-0.134183,1.340100,0.741324,0.554025,1.0
7,0.060123,-2.850018,-1.808885,1.649850,0.827631,-0.681723,3.049334,-1.560887,-0.029848,0.037102,0.0
8,1.735121,4.266722,0.767749,-2.012636,2.538945,-4.960979,2.908603,-6.451668,2.723697,0.785976,0.0
9,0.436313,1.826231,-0.232626,-0.570040,1.101895,0.658344,-0.079847,-2.158960,0.985347,0.426746,0.0


In [12]:
optimal_point(df2, model=model, desired_class=0, original_class=1, resolution=15, chosen_row=1, point_epsilon=0.1, epsilon=0.07)

Class counts:
 10
1    1005
0    1005
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
Not enough memory to generate a grid with 15^10 number of points


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


In [13]:
optimal_point(df2, model=model, desired_class=0, original_class=1, resolution=10, chosen_row=1, point_epsilon=0.1, epsilon=0.07)

Class counts:
 10
1    1005
0    1005
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
Not enough memory to generate a grid with 10^10 number of points


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


In [14]:
optimal_point(df3, model=model, desired_class=0, original_class=1, resolution=15, chosen_row=1, point_epsilon=0.1, epsilon=0.07)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 50
1    1001
0    1001
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...


ValueError: Maximum allowed dimension exceeded

In [15]:
optimal_point(df3, model=model, desired_class=0, original_class=1, resolution=10, chosen_row=1, point_epsilon=0.1, epsilon=0.07)

Class counts:
 50
1    1001
0    1001
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


ValueError: Maximum allowed dimension exceeded

# Experiments of the Computation Costs for SSBA method Using GPU (Logistic Regression)


$\textbf{ Logistic Regression with SSBA Method }$ -- 50 features

$T = 1,000,000$, 6.2 seconds runtime, $689,574$ boundary points found

$T = 250,000$ $, 2.2 seconds runtime, $500,000$ boundary points found

$T = 100,000$, 0.9 seconds runtime, $100,000$ boundary points found

$T = 10,000$, 0.1 seconds runtime, $10,000$ boundary points found

$\textbf{ Logistic Regression with SSBA Method }$ -- 10 features

$T = 1,000,000$, ~2.2 seconds runtime, $711,436$ boundary points found

$T = 250,000$, ~0.9 seconds runtime, $250,000$ boundary points found

$T = 100,000$, ~0.4 seconds runtime, $100,000$ boundary points found

$T = 10,000$, ~0.1 seconds runtime, $10,000$ boundary points found


$\textbf{ Logistic Regression with SSBA Method }$ -- 2 features

$T = 1,000,000$, ~2.4 seconds runtime, $793,845$ boundary points found

$T = 250,000$, ~0.7 seconds runtime, $250,000$ boundary points found

$T = 100,000$, ~0.3 seconds runtime, $100,000$ boundary points found

$T = 10,000$, ~0.0 seconds runtime, $10,000$ boundary points found

$T = 1,500$, ~0.0 seconds runtime, $1,500$ boundary points found

$T = 500$, ~0.0 seconds runtime, $500$ boundary points found

In [16]:
from files.binary_search_optimal_point import optimal_point

In [17]:
optimal_point(df1, model=model, desired_class=0, original_class=1, threshold=500, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(500, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01086188 1.04264849]]
[[-1.17285622 -0.03193558]]


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


[np.float64(0.0015030385232364107), np.float64(1.1264895731106561)]

In [18]:
optimal_point(df1, model=model, desired_class=0, original_class=1, threshold=1500, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(1500, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01086307 1.03152629]]
[[-1.17123926 -0.04791937]]


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


[np.float64(0.0031361662356048647), np.float64(1.1103570686441802)]

In [19]:
optimal_point(df1, model=model, desired_class=0, original_class=1, threshold=10000,  chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(10000, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01086398 1.03174952]]
[[-1.17001351 -0.04743782]]


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


[np.float64(0.0043741678361628455), np.float64(1.1108432174395366)]

In [20]:
optimal_point(df1, model=model, desired_class=0, original_class=1, threshold=100000,  chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100000, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01086502 1.02418796]]
[[-1.16860428 -0.0721871 ]]


[np.float64(0.0057974944660421635), np.float64(1.0858539989768687)]

In [21]:
optimal_point(df1, model=model, desired_class=0, original_class=1, threshold=250000,  chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(250000, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01086398 1.03174952]]
[[-1.17001351 -0.04743782]]


[np.float64(0.0043741678361628455), np.float64(1.1108432174395366)]

In [22]:
optimal_point(df1, model=model, desired_class=0, original_class=1, threshold=1000000,  chosen_row=1, point_epsilon=1e-4, epsilon=1e-3)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 2
1    1000
0    1000
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(793845, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.00185492 1.02077407]]
[[-1.17187423 -0.05162185]]


[np.float64(0.013050724564066973), np.float64(1.1070929121254551)]

In [19]:
optimal_point(df2, model=model, desired_class=0, original_class=1, threshold=10000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

Class counts:
 10
1    1005
0    1005
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(10000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.02729977 1.01526476 0.99263632 1.01480599 1.00672334 1.00033393
  1.00202299 1.01605727 1.01787145 1.01256596]]
[[-0.05938227 -0.19284177  0.05716738 -0.21115423  0.307241    0.10348921
   0.12561382 -0.16774169 -0.12931178 -0.39461488]]


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


[np.float64(1.2813530480728388),
 np.float64(0.7471710632588614),
 np.float64(3.1145287451993857),
 np.float64(-0.1546138606265391),
 np.float64(-0.30201012178874953),
 np.float64(-0.8819969784732707),
 np.float64(0.44971862490110337),
 np.float64(0.6832840907073147),
 np.float64(1.609413070545779),
 np.float64(1.669749072295262)]

In [20]:
optimal_point(df2, model=model, desired_class=0, original_class=1, threshold=100000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 10
1    1005
0    1005
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.10535072 1.01383935 1.03012942 1.01441507 1.00617286 1.08904428
  0.86467458 1.01584238 1.02079412 1.01311026]]
[[-0.01159247 -0.2640651  -0.05117532 -0.2297617   0.26290475 -0.01377765
   0.00594992 -0.17387486 -0.09456945 -0.32573135]]


[np.float64(1.3295426862024735),
 np.float64(0.6752369177547147),
 np.float64(3.0050651230283156),
 np.float64(-0.17340701830114585),
 np.float64(-0.346789182220165),
 np.float64(-1.0005252128751565),
 np.float64(0.32899543559661043),
 np.float64(0.677089806279022),
 np.float64(1.6444999034668655),
 np.float64(1.7393208956966077)]

In [21]:
optimal_point(df2, model=model, desired_class=0, original_class=1, threshold=250000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 10
1    1005
0    1005
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(250000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.07355226 1.01650651 1.02523848 1.01436357 1.00647331 1.01670969
  1.00077519 1.01357508 1.00104457 1.01736361]]
[[-0.01689243 -0.15622906 -0.06727959 -0.2324618   0.28538742 -0.15152854
   0.1084874  -0.28351106  0.11178079 -0.13816092]]


[np.float64(1.324221527350697),
 np.float64(0.78414865084697),
 np.float64(2.9888047006260083),
 np.float64(-0.176134061267755),
 np.float64(-0.3240819876296826),
 np.float64(-1.1395812828187764),
 np.float64(0.4324221925646943),
 np.float64(0.5663595093001842),
 np.float64(1.852933387365994),
 np.float64(1.928762777949529)]

In [22]:
optimal_point(df2, model=model, desired_class=0, original_class=1, threshold=1000000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 10
1    1005
0    1005
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(711436, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.07355226 1.01650651 1.02523848 1.01436357 1.00647331 1.01670969
  1.00077519 1.01357508 1.00104457 1.01736361]]
[[-0.01689243 -0.15622906 -0.06727959 -0.2324618   0.28538742 -0.15152854
   0.1084874  -0.28351106  0.11178079 -0.13816092]]


[np.float64(1.324221527350697),
 np.float64(0.78414865084697),
 np.float64(2.9888047006260083),
 np.float64(-0.176134061267755),
 np.float64(-0.3240819876296826),
 np.float64(-1.1395812828187764),
 np.float64(0.4324221925646943),
 np.float64(0.5663595093001842),
 np.float64(1.852933387365994),
 np.float64(1.928762777949529)]

In [23]:
optimal_point(df3, model=model, desired_class=0, original_class=1, threshold=10000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

Class counts:
 50
1    1001
0    1001
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(10000, 50)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01037082 1.00958436 1.00972634 1.00541612 1.01076848 1.00983784
  1.01035458 1.0108802  1.00988798 1.01042748 1.01250418 1.00784728
  1.01061702 1.01037793 1.00808982 1.00911111 1.01035653 1.01041808
  1.01054178 1.01040066 1.00967898 1.01052102 1.00951753 1.00880109
  1.00944602 1.00538008 1.0104699  1.03230827 1.01028178 1.00979553
  1.01040033 1.01037245 1.00969037 1.01245074 1.00976025 1.00937991
  1.01016229 1.00985392 1.00965079 1.00934566 1.01042756 1.01108555
  1.00959395 1.00967192 1.01142143 1.01422801 1.02279576 1.00966673
  1.00973968 1.00817211]]
[[-2.72472197  2.42896813  3.68966393  0.21933713 -1.3152751   6.22740142
  -2.84943149 -1.14847099  9.01513052 -2.36368712 

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


[np.float64(8.290932063472475),
 np.float64(3.7579453881885767),
 np.float64(-0.4365160069280698),
 np.float64(7.282676644214999),
 np.float64(8.858742484321501),
 np.float64(1.0717022435734762),
 np.float64(-2.009092158696899),
 np.float64(5.234314070895903),
 np.float64(-5.394783390585642),
 np.float64(-5.014600160081644),
 np.float64(0.9755495491115459),
 np.float64(2.289169121756455),
 np.float64(-4.132677224516502),
 np.float64(-3.4007249257855237),
 np.float64(1.027547734075198),
 np.float64(1.2481603351921242),
 np.float64(-1.6802207877493687),
 np.float64(-0.025584521380677572),
 np.float64(-4.370313399776749),
 np.float64(-3.905826720162301),
 np.float64(-0.9840505388632614),
 np.float64(7.072072707891234),
 np.float64(0.2503045575525511),
 np.float64(1.4923693463691319),
 np.float64(-0.009203988159701249),
 np.float64(1.9428691642679004),
 np.float64(1.2591398024063025),
 np.float64(1.7334964740258014),
 np.float64(3.7920880404170543),
 np.float64(-1.5407527230796623),
 np.fl

In [24]:
optimal_point(df3, model=model, desired_class=0, original_class=1, threshold=100000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 50
1    1001
0    1001
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100000, 50)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01024303 1.01336232 1.00957532 1.01043767 1.01033117 1.00984082
  0.96124046 1.01027334 1.00978937 1.00376643 1.0056014  1.01092432
  1.00974171 1.0107787  1.0093797  1.01087059 1.00953054 1.0107828
  1.00873957 1.00945339 1.00977284 1.01064132 1.00936313 1.01264721
  1.00726442 1.01087759 1.00642941 1.00965034 1.01020549 1.009447
  1.01015697 1.01017106 1.00701244 1.01056156 1.01186838 1.00889362
  1.01027234 1.0097269  1.01025938 1.01052338 1.01224907 1.01074029
  1.00970063 1.0095788  1.20147884 1.01041222 1.03407954 1.00980219
  1.0095747  1.00929185]]
[[-4.15679094e+00 -3.01388083e-01  2.37728590e+00 -2.30868013e+00
  -3.05075113e+00  6.34385469e+00  1.97138931e-02 -3.6960886

[np.float64(6.844542524433441),
 np.float64(1.000281835880676),
 np.float64(-1.7620176726724863),
 np.float64(4.729374186221488),
 np.float64(7.105912132135625),
 np.float64(1.189320048599245),
 np.float64(0.888793790198757),
 np.float64(2.661220887922471),
 np.float64(-9.65808350583552),
 np.float64(-2.464633390384614),
 np.float64(1.6148292152206312),
 np.float64(0.7116742022952347),
 np.float64(1.4699673299072669),
 np.float64(-2.0115475952262396),
 np.float64(2.138038363225559),
 np.float64(-1.0711755506445897),
 np.float64(3.353882835175872),
 np.float64(1.1112495965991207),
 np.float64(-1.678109996931357),
 np.float64(0.5064375360008537),
 np.float64(0.3288796998407051),
 np.float64(7.439327028884444),
 np.float64(-0.26229865039209854),
 np.float64(0.2561614122208343),
 np.float64(-1.4777083634800954),
 np.float64(0.5596692305916615),
 np.float64(3.7157232940125478),
 np.float64(4.6966291534995355),
 np.float64(2.448194080940368),
 np.float64(-4.68506941181309),
 np.float64(-3.19

In [27]:
optimal_point(df3, model=model, desired_class=0, original_class=1, threshold=250000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 50
1    1001
0    1001
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(250000, 50)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01016285 1.00978172 1.00959691 1.01018086 1.01023564 1.00970167
  1.00906782 1.01079172 1.0098747  1.00959531 1.01026077 1.01026997
  1.00938092 1.00640707 1.00970065 1.00934682 1.00924954 1.01022833
  1.00923392 1.00870558 1.00926939 1.01027767 1.15156955 1.01034185
  1.00956214 1.00951857 1.0105032  1.00839421 1.00109924 1.00927276
  1.01043169 1.02007724 1.01443231 1.00955004 1.01370924 1.0106066
  1.01030338 1.00960733 1.01055797 1.01057371 1.01030576 1.00951209
  1.00939173 1.01136188 1.00956361 1.00975567 1.00894374 1.00980913
  1.00971697 1.01461338]]
[[-6.20292604  4.62606574  2.50464325 -5.58541919 -4.28726181  3.38449357
   1.08248051 -1.27669899  8.05934393  2.49472903 

[np.float64(4.777946158422404),
 np.float64(5.977013774993333),
 np.float64(-1.633386771326594),
 np.float64(1.4198679926680526),
 np.float64(5.857036441354809),
 np.float64(-1.799634539635242),
 np.float64(1.9621402508788526),
 np.float64(5.104803874827192),
 np.float64(-6.3601278325124575),
 np.float64(-0.10759901397252092),
 np.float64(-2.529005757912265),
 np.float64(-1.9632770808852495),
 np.float64(-0.8316267908049884),
 np.float64(-0.417627743908126),
 np.float64(3.901221032034706),
 np.float64(1.6622996740134295),
 np.float64(2.540280549692748),
 np.float64(-2.053345972880043),
 np.float64(-1.1558581530754275),
 np.float64(-0.5717267472380506),
 np.float64(-2.765493192349362),
 np.float64(5.3561665033104955),
 np.float64(-1.871378076128801),
 np.float64(-2.3425798262012845),
 np.float64(0.4791411442680009),
 np.float64(3.840950295447394),
 np.float64(1.4028051117464444),
 np.float64(2.4145134094968776),
 np.float64(7.526944396048299),
 np.float64(-5.127021997464764),
 np.float6

In [26]:
optimal_point(df3, model=model, desired_class=0, original_class=1, threshold=1000000, chosen_row=1, point_epsilon=1e-3, epsilon=0.01)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:426: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 50
1    1001
0    1001
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(689574, 50)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01028144 1.0093257  1.00898715 1.01049448 1.01030976 1.00975198
  1.00953466 1.01023855 1.00988581 1.010713   1.0105713  1.01039549
  1.00847385 1.01064309 1.00947162 1.012031   1.01026245 1.01042152
  1.01615662 1.01080378 1.00968541 1.01019558 1.01194125 1.01056519
  1.00914584 1.00945694 1.01062109 1.00942733 1.01028276 1.00973846
  1.01049841 1.01049315 1.00900146 1.01050555 1.00975036 1.0091618
  1.01082661 1.00866522 1.00862605 1.00936586 1.01027968 1.009335
  1.01066487 1.00980168 1.00927085 1.01501322 1.01126741 1.00975182
  1.0095769  1.0082031 ]]
[[-3.5897048   1.49685165  0.99619028 -2.04353689 -3.2615739   4.07127148
   2.169471   -4.23488437  8.84388891 -1.4175559  -1

[np.float64(7.417299486882551),
 np.float64(2.816508002888985),
 np.float64(-3.1569236570316432),
 np.float64(4.99716880573836),
 np.float64(6.892981152568453),
 np.float64(-1.1059889100135818),
 np.float64(3.0600001700919863),
 np.float64(2.117037191843826),
 np.float64(-5.567737409802822),
 np.float64(-4.059007911928413),
 np.float64(-0.4026725307685768),
 np.float64(-0.7640593736475993),
 np.float64(-1.810982169563153),
 np.float64(-2.287798232907855),
 np.float64(2.4241158864001062),
 np.float64(-0.4017118413774091),
 np.float64(-2.7058285658347376),
 np.float64(-0.005666718631546441),
 np.float64(-2.65313458959901),
 np.float64(-2.628942111898315),
 np.float64(-0.9190809758835359),
 np.float64(3.814197741986005),
 np.float64(-2.3895181021104617),
 np.float64(-1.1633516702011095),
 np.float64(-0.6563389821927528),
 np.float64(3.600485716572961),
 np.float64(1.7875913089733722),
 np.float64(3.56055177408217),
 np.float64(3.8046594427466522),
 np.float64(-2.629357903351338),
 np.floa